In [ ]:
import basedosdados as bd
import pandas as pd

# Consulta SQL para buscar chamados do dia 01/04/2023
query_chamados = (
    """
    SELECT
        tb_chamado.id_chamado,
        tb_chamado.data_inicio,
        tb_chamado.tipo,
        tb_chamado.id_bairro,
        tb_bairro.nome AS bairro,
        tb_bairro.subprefeitura
    FROM datario.adm_central_atendimento_1746.chamado tb_chamado
    LEFT JOIN datario.dados_mestres.bairro tb_bairro
        ON tb_chamado.id_bairro = tb_bairro.id_bairro
    WHERE DATE(tb_chamado.data_inicio) = '2023-04-01'
    """
)
df_1_5 = bd.read_sql(query=query_chamados, billing_project_id="secc-rj")

# Consulta SQL para buscar chamados de "Perturbação do Sossego" entre 2022 e 2024
query_perturbacao = """
    SELECT
        tb_chamado.id_chamado,
        DATE(tb_chamado.data_inicio) AS data_inicio,
        tb_chamado.id_subtipo
    FROM datario.adm_central_atendimento_1746.chamado tb_chamado
    WHERE tb_chamado.id_subtipo = '5071'
      AND DATE(tb_chamado.data_inicio) BETWEEN '2022-01-01' AND '2024-12-31'
"""

df_6_10 = bd.read_sql(query=query_perturbacao, billing_project_id="secc-rj")

# Consulta SQL para buscar eventos turísticos e hoteleiros
df_eventos = bd.read_sql(
    query="SELECT * FROM `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos`",
    billing_project_id="secc-rj",
)

# Limpeza e preparação dos dados de eventos
df_eventos = df_eventos.dropna(subset=["data_inicial", "data_final"])
df_eventos["data_inicial"] = pd.to_datetime(df_eventos["data_inicial"]).dt.date
df_eventos["data_final"]   = pd.to_datetime(df_eventos["data_final"]).dt.date
df_eventos = df_eventos[df_eventos["evento"].isin(["Réveillon", "Carnaval", "Rock in Rio"])]

# Criar campo com o número de dias de cada evento
df_eventos["dias_evento"] = (df_eventos["data_final"] - df_eventos["data_inicial"]) \
    .apply(lambda x: x.days) + 1

# Converter a coluna data_inicio para tipo date
df_6_10["data_inicio"] = pd.to_datetime(df_6_10["data_inicio"]).dt.date

# Dicionário de respostas
respostas = {}


Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|


# Respostas das Perguntas 1 a 5

In [1]:
# Pergunta 01: Quantos chamados foram abertos no dia 01/04/2023?
resp_1 = df_1_5["id_chamado"].count()
respostas["pergunta_01"] = resp_1
print(f'1) Quantos chamados foram abertos no dia 01/04/2023?\n{resp_1}')

NameError: name 'df_1_5' is not defined

In [ ]:

# Pergunta 02: Qual o tipo de chamado que teve mais teve chamados abertos no dia 01/04/2023?
if not df_1_5.empty:
    tipo_chamado_count = (
        df_1_5.groupby("tipo")["id_chamado"]
        .count()
        .sort_values(ascending=False)
    )
    tipo_chamado = tipo_chamado_count.index[0]
    respostas["pergunta_02"] = (tipo_chamado, tipo_chamado_count.iloc[0])
else:
    respostas["pergunta_02"] = None
print(f'2) Qual o tipo de chamado que teve mais teve chamados abertos no dia 01/04/2023?\n"{tipo_chamado}" com {tipo_chamado_count.iloc[0]} chamadas')

2) Qual o tipo de chamado que teve mais teve chamados abertos no dia 01/04/2023?
"Estacionamento irregular" com 373 chamadas


In [ ]:
# Pergunta 03: Quais os nomes dos 3 bairros que mais tiveram chamados abertos nesse dia?
if not df_1_5.empty:
    bairro_chamado_count = (
        df_1_5.groupby("bairro")["id_chamado"]
        .count()
        .sort_values(ascending=False)
        .head(3)
    )
    resp_3 = list(bairro_chamado_count.items())
    respostas["pergunta_03"] = resp_3
else:
    respostas["pergunta_03"] = []

print(f'3) Quais os nomes dos 3 bairros que mais tiveram chamados abertos nesse dia?')
for i, j in resp_3:
    print(f'"{i}" com {j} chamadas')

3) Quais os nomes dos 3 bairros que mais tiveram chamados abertos nesse dia?
"Campo Grande" com 124 chamadas
"Tijuca" com 96 chamadas
"Barra da Tijuca" com 60 chamadas


In [ ]:
# Pergunta 04: Qual o nome da subprefeitura com mais chamados abertos nesse dia?
if not df_1_5.empty:
    subprefeitura_chamado_count = (
        df_1_5.groupby("subprefeitura")["id_chamado"]
        .count()
        .sort_values(ascending=False)
    )
    subprefeitura_chamado = subprefeitura_chamado_count.index[0]
    respostas["pergunta_04"] = (subprefeitura_chamado,
                                subprefeitura_chamado_count.iloc[0])
else:
    respostas["pergunta_04"] = None
print(f'4) Qual o nome da subprefeitura com mais chamados abertos nesse dia? \n "{subprefeitura_chamado}" com {subprefeitura_chamado_count.iloc[0]} chamadas')

4) Qual o nome da subprefeitura com mais chamados abertos nesse dia? 
 "Zona Norte" com 534 chamadas


In [ ]:
# Pergunta 05: Existe algum chamado aberto nesse dia que não foi associado a um bairro
# ou subprefeitura na tabela de bairros? Se sim, por que isso acontece?
chamados_null = df_1_5[
    df_1_5["id_bairro"].isnull() | df_1_5["subprefeitura"].isnull()
]["id_chamado"].count()
respostas["pergunta_05"] = chamados_null
print(f"""
5) Existe algum chamado aberto nesse dia que não foi associado a um bairro ou subprefeitura na tabela de bairros?
Se sim, por que isso acontece?
Há {chamados_null} chamadas abertas sem associação a bairro e subprefeitura.
Como não possuem vínculo com um id_bairro ou subprefeitura,
logo podem corresponder a tipos independentes do espaço geográfico.
Um exemplo seriam as demandas do atendimento ao cidadão, como dúvidas,
sugestões ou notificações sobre erros e problemas em serviços digitais
da prefeitura que não estão vinculadas a um local específico.""")


5) Existe algum chamado aberto nesse dia que não foi associado a um bairro ou subprefeitura na tabela de bairros?
Se sim, por que isso acontece?
Há 131 chamadas abertas sem associação a bairro e subprefeitura.
Como não possuem vínculo com um id_bairro ou subprefeitura,
logo podem corresponder a tipos independentes do espaço geográfico.
Um exemplo seriam as demandas do atendimento ao cidadão, como dúvidas,
sugestões ou notificações sobre erros e problemas em serviços digitais
da prefeitura que não estão vinculadas a um local específico.


# Respostas das Perguntas 6 a 10

In [ ]:
# Pergunta 06: Quantos chamados com o subtipo "Perturbação do sossego" 
# foram abertos desde 01/01/2022 até 31/12/2023 (incluindo extremidades)?
resp_6 = df_6_10["id_chamado"].nunique()  # .count() também seria aceitável caso não haja duplicados
respostas["pergunta_06"] = resp_6

print(f"""
6) Quantos chamados de "Perturbação do sossego" (id_subtipo=5071) foram abertos 
no período de 01/01/2022 até 31/12/2024?
R: Foram {resp_6} chamados no total.
""")


6) Quantos chamados de "Perturbação do sossego" (id_subtipo=5071) foram abertos 
no período de 01/01/2022 até 31/12/2024?
R: Foram 56785 chamados no total.



In [ ]:
# Pergunta 07: Selecione os chamados com esse subtipo que foram abertos durante
# os eventos contidos na tabela de eventos (Reveillon, Carnaval e Rock in Rio).

# Faremos um "merge vetorizado" (cross join) e filtraremos pela condição de data
df_cross = df_6_10.assign(key=1).merge(
    df_eventos[["evento", "data_inicial", "data_final"]].assign(key=1),
    on="key",
    how="outer"
)

# Filtrar chamados que caem no intervalo do evento
chamados_eventos = df_cross[
    (df_cross["data_inicio"] >= df_cross["data_inicial"]) &
    (df_cross["data_inicio"] <= df_cross["data_final"])
].copy()

# Remover colunas auxiliares e possíveis duplicidades
chamados_eventos.drop(columns=["key"], inplace=True)
chamados_eventos.drop_duplicates(subset=["id_chamado", "evento"], inplace=True)

respostas["pergunta_07"] = chamados_eventos

print("""
7) Lista de chamados (id_subtipo=5071) que foram abertos durante 
os eventos (Réveillon, Carnaval, Rock in Rio):
""")
print(chamados_eventos.head())


7) Lista de chamados (id_subtipo=5071) que foram abertos durante 
os eventos (Réveillon, Carnaval, Rock in Rio):

       id_chamado data_inicio id_subtipo     evento data_inicial  data_final
180287   18076121  2022-12-30       5071  Réveillon   2022-12-30  2023-01-01
180563   18077630  2022-12-30       5071  Réveillon   2022-12-30  2023-01-01
180713   18075958  2022-12-30       5071  Réveillon   2022-12-30  2023-01-01
180767   18078205  2022-12-30       5071  Réveillon   2022-12-30  2023-01-01
181061   18076125  2022-12-30       5071  Réveillon   2022-12-30  2023-01-01


In [ ]:
# Pergunta 08: Quantos chamados desse subtipo foram abertos em cada evento?
if not chamados_eventos.empty:
    qtd_chamados_evento = (
        chamados_eventos.groupby("evento")["id_chamado"]
        .count()
        .sort_values(ascending=False)
    )
    respostas["pergunta_08"] = qtd_chamados_evento
    print("\n8) Quantos chamados desse subtipo foram abertos em cada evento?\n")
    for evento, qtd in qtd_chamados_evento.items():
        print(f' - Evento "{evento}": {qtd} chamados')
else:
    respostas["pergunta_08"] = None
    print("\n8) Não houve chamados para nenhum evento.\n")


8) Quantos chamados desse subtipo foram abertos em cada evento?

 - Evento "Rock in Rio": 946 chamados
 - Evento "Carnaval": 252 chamados
 - Evento "Réveillon": 147 chamados


In [ ]:
# Pergunta 09: Qual evento teve a maior média diária de chamados abertos desse subtipo?
if not chamados_eventos.empty:
    # Soma de chamados por evento
    qtd_chamados_evento = chamados_eventos.groupby("evento")["id_chamado"].count()

    # Soma de dias para cada evento (caso haja duplicação de períodos, agrupar por evento)
    soma_dias_evento = (
        df_eventos.drop_duplicates(subset=["evento", "data_inicial", "data_final"])
        .groupby("evento")["dias_evento"]
        .sum()
    )

    # Média diária por evento = total de chamados no evento / total de dias do evento
    med_dias_chamados = (qtd_chamados_evento / soma_dias_evento).sort_values(ascending=False)

    # Qual evento teve a maior média
    maior_media_evento = med_dias_chamados.head(1)
    respostas["pergunta_09"] = maior_media_evento

    print("\n9) Qual evento teve a maior média diária de chamados abertos?\n")
    print(f"O evento '{maior_media_evento.index[0]}' apresentou a maior média diária de "
          f"{maior_media_evento.iloc[0]:.2f} chamados.")
else:
    respostas["pergunta_09"] = None
    print("\n9) Não foi possível calcular a média diária por evento, pois não há dados.\n")


9) Qual evento teve a maior média diária de chamados abertos?

O evento 'Rock in Rio' apresentou a maior média diária de 135.14 chamados.


In [ ]:
# Pergunta 10: Compare as médias diárias de chamados abertos desse subtipo
# durante os eventos específicos (Reveillon, Carnaval e Rock in Rio) e
# a média diária de chamados abertos desse tipo considerando todo o
# período de 01/01/2022 até 31/12/2023.
if not df_6_10.empty:
    # Dias totais no período (do 1º chamado ao último chamado do dataframe)
    data_min = df_6_10["data_inicio"].min()
    data_max = df_6_10["data_inicio"].max()
    dias_periodo = (data_max - data_min).days + 1

    # Média diária total (chamados / dias)
    media_diaria_total = resp_6 / dias_periodo if dias_periodo else 0

    # Se já calculamos med_dias_chamados no passo anterior:
    if 'med_dias_chamados' in locals():
        df_med_chamados = med_dias_chamados.to_frame(name="media_diaria_evento")
        df_med_chamados["media_diaria_total_periodo"] = media_diaria_total
        respostas["pergunta_10"] = df_med_chamados
    else:
        # Caso não haja dados de eventos, salvamos apenas a média total
        respostas["pergunta_10"] = media_diaria_total

    print(f"""
10) Comparando as médias diárias de chamados (id_subtipo=5071) nos eventos 
(Réveillon, Carnaval, Rock in Rio) versus a média diária em todo período 
01/01/2022 a 31/12/2024:
""")

    if isinstance(respostas["pergunta_10"], pd.DataFrame):
        for evento, row in respostas["pergunta_10"].iterrows():
            print(f'Evento "{evento}": média diária = {row["media_diaria_evento"]:.2f} chamados; '
                  f'média total do período = {row["media_diaria_total_periodo"]:.2f} chamados.')
    else:
        print("Não há dados de eventos para comparar. "
              f"Apenas a média diária total do período: {respostas['pergunta_10']:.2f}")
else:
    respostas["pergunta_10"] = None
    print("\n10) Não há dados de chamados no dataframe para comparar.\n")


10) Comparando as médias diárias de chamados (id_subtipo=5071) nos eventos 
(Réveillon, Carnaval, Rock in Rio) versus a média diária em todo período 
01/01/2022 a 31/12/2024:

Evento "Rock in Rio": média diária = 135.14 chamados; média total do período = 51.81 chamados.
Evento "Carnaval": média diária = 28.00 chamados; média total do período = 51.81 chamados.
Evento "Réveillon": média diária = 24.50 chamados; média total do período = 51.81 chamados.


In [ ]:
# Print de todas as respostas esquematizadas
for pergunta, resposta in respostas.items():
    print(f"Resposta da {pergunta}:")
    print(resposta)
    print("-" * 50)

Resposta da pergunta_01:
1903
--------------------------------------------------
Resposta da pergunta_02:
('Estacionamento irregular', 373)
--------------------------------------------------
Resposta da pergunta_03:
[('Campo Grande', 124), ('Tijuca', 96), ('Barra da Tijuca', 60)]
--------------------------------------------------
Resposta da pergunta_04:
('Zona Norte', 534)
--------------------------------------------------
Resposta da pergunta_05:
131
--------------------------------------------------
Resposta da pergunta_06:
56785
--------------------------------------------------
Resposta da pergunta_07:
       id_chamado data_inicio id_subtipo     evento data_inicial  data_final
180287   18076121  2022-12-30       5071  Réveillon   2022-12-30  2023-01-01
180563   18077630  2022-12-30       5071  Réveillon   2022-12-30  2023-01-01
180713   18075958  2022-12-30       5071  Réveillon   2022-12-30  2023-01-01
180767   18078205  2022-12-30       5071  Réveillon   2022-12-30  2023-01-01
